# Notebook 4: Dimensionality Reduction (PCA)

**Purpose**: Apply PCA/Kernel PCA to reduce features to ≤10 dimensions for QML compatibility.

**Inputs**:
- `feature_engineered_train.csv` from Notebook 3
- `feature_engineered_test.csv` from Notebook 3

**Outputs**:
- `pca_train.csv` → `data/features/`
- `pca_test.csv` → `data/features/`
- `pca_model.pkl` → `models/`

---

In [ ]:
# Imports
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, KernelPCA
import joblib

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Paths
BASE_DIR = Path('.').resolve().parent
FEATURES_DIR = BASE_DIR / 'data' / 'features'
MODELS_DIR = BASE_DIR / 'models'
RESULTS_DIR = BASE_DIR / 'results'
FIGURES_DIR = BASE_DIR / 'figures'

# Configuration
TARGET_COLUMN = 'Class'
MAX_COMPONENTS = 10  # Maximum features for QML

print(f"Random Seed: {RANDOM_SEED}")
print(f"Max Components: {MAX_COMPONENTS}")

## 1. Load Feature-Engineered Data

In [ ]:
# Load data
train_df = pd.read_csv(FEATURES_DIR / 'feature_engineered_train.csv')
test_df = pd.read_csv(FEATURES_DIR / 'feature_engineered_test.csv')

print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

In [ ]:
# Separate features and target
X_train = train_df.drop(columns=[TARGET_COLUMN])
y_train = train_df[TARGET_COLUMN]

X_test = test_df.drop(columns=[TARGET_COLUMN])
y_test = test_df[TARGET_COLUMN]

print(f"Features before PCA: {X_train.shape[1]}")

## 2. Variance Analysis (Full PCA)

In [ ]:
# Fit full PCA to analyze variance explained
pca_full = PCA(random_state=RANDOM_SEED)
pca_full.fit(X_train)

# Calculate cumulative variance explained
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"Total components: {len(explained_variance)}")
print(f"\nVariance explained by first {MAX_COMPONENTS} components: {cumulative_variance[MAX_COMPONENTS-1]*100:.2f}%")

In [ ]:
# Visualize variance explained
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual variance
n_show = min(20, len(explained_variance))
axes[0].bar(range(1, n_show+1), explained_variance[:n_show], color='steelblue')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance Explained Ratio')
axes[0].set_title('Variance Explained by Component')
axes[0].axvline(x=MAX_COMPONENTS, color='red', linestyle='--', label=f'n={MAX_COMPONENTS}')
axes[0].legend()

# Cumulative variance
axes[1].plot(range(1, n_show+1), cumulative_variance[:n_show], 'bo-')
axes[1].axhline(y=0.95, color='green', linestyle='--', label='95% variance')
axes[1].axhline(y=0.99, color='orange', linestyle='--', label='99% variance')
axes[1].axvline(x=MAX_COMPONENTS, color='red', linestyle='--', label=f'n={MAX_COMPONENTS}')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance Explained')
axes[1].set_title('Cumulative Variance Explained')
axes[1].legend()
axes[1].set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'pca_variance_explained.png', dpi=150)
plt.show()

## 3. Optimal Component Selection

In [ ]:
# Find minimum components for different variance thresholds
thresholds = [0.90, 0.95, 0.99]

print("Components needed for variance thresholds:")
for thresh in thresholds:
    n_components = np.argmax(cumulative_variance >= thresh) + 1
    print(f"  {thresh*100:.0f}% variance: {n_components} components")

# We'll use MAX_COMPONENTS as our target
n_components = MAX_COMPONENTS
print(f"\n✅ Using {n_components} components (captures {cumulative_variance[n_components-1]*100:.2f}% variance)")

## 4. Apply PCA

In [ ]:
# Fit PCA with selected number of components
pca = PCA(n_components=n_components, random_state=RANDOM_SEED)

# Fit on training data, transform both
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Create column names
pca_columns = [f'PC{i+1}' for i in range(n_components)]

# Convert to DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=pca_columns, index=X_train.index)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=pca_columns, index=X_test.index)

print(f"Training PCA shape: {X_train_pca_df.shape}")
print(f"Test PCA shape: {X_test_pca_df.shape}")

In [ ]:
# Display component statistics
print("PCA Component Statistics (Training):")
print(X_train_pca_df.describe())

## 5. Visualize Principal Components

In [ ]:
# 2D scatter plot of first two components
plt.figure(figsize=(10, 8))

colors = ['#2ecc71', '#e74c3c']
labels = ['Normal', 'Anomaly']

for i, label in enumerate([0, 1]):
    mask = y_train == label
    plt.scatter(
        X_train_pca_df.loc[mask, 'PC1'],
        X_train_pca_df.loc[mask, 'PC2'],
        c=colors[i],
        label=labels[i],
        alpha=0.6,
        s=50
    )

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
plt.title('PCA: First Two Principal Components')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'pca_2d_scatter.png', dpi=150)
plt.show()

In [ ]:
# 3D scatter plot
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

for i, label in enumerate([0, 1]):
    mask = y_train == label
    ax.scatter(
        X_train_pca_df.loc[mask, 'PC1'],
        X_train_pca_df.loc[mask, 'PC2'],
        X_train_pca_df.loc[mask, 'PC3'],
        c=colors[i],
        label=labels[i],
        alpha=0.6,
        s=50
    )

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('PCA: First Three Principal Components')
ax.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'pca_3d_scatter.png', dpi=150)
plt.show()

In [ ]:
# Component distributions by class
fig, axes = plt.subplots(2, 5, figsize=(16, 6))
axes = axes.flatten()

for i, col in enumerate(pca_columns):
    ax = axes[i]
    for j, label in enumerate([0, 1]):
        mask = y_train == label
        ax.hist(
            X_train_pca_df.loc[mask, col],
            bins=30,
            alpha=0.5,
            color=colors[j],
            label=labels[j]
        )
    ax.set_title(col)
    ax.legend(fontsize=8)

plt.suptitle('Component Distributions by Class', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'pca_component_distributions.png', dpi=150)
plt.show()

## 6. Save Outputs

In [ ]:
# Combine with target
train_pca_final = X_train_pca_df.copy()
train_pca_final[TARGET_COLUMN] = y_train.values

test_pca_final = X_test_pca_df.copy()
test_pca_final[TARGET_COLUMN] = y_test.values

print(f"Final training shape: {train_pca_final.shape}")
print(f"Final test shape: {test_pca_final.shape}")

In [ ]:
# Save PCA data
train_path = FEATURES_DIR / 'pca_train.csv'
test_path = FEATURES_DIR / 'pca_test.csv'

train_pca_final.to_csv(train_path, index=False)
test_pca_final.to_csv(test_path, index=False)

print(f"✅ Saved PCA training data to: {train_path}")
print(f"✅ Saved PCA test data to: {test_path}")

In [ ]:
# Save PCA model
pca_path = MODELS_DIR / 'pca_model.pkl'
joblib.dump(pca, pca_path)
print(f"✅ Saved PCA model to: {pca_path}")

In [ ]:
# Save PCA metadata
pca_metadata = {
    "random_seed": RANDOM_SEED,
    "n_components": n_components,
    "original_features": X_train.shape[1],
    "variance_explained": {
        "per_component": [round(v, 4) for v in pca.explained_variance_ratio_],
        "cumulative": round(sum(pca.explained_variance_ratio_), 4)
    },
    "component_names": pca_columns
}

with open(RESULTS_DIR / 'pca_metadata.json', 'w') as f:
    json.dump(pca_metadata, f, indent=2)

print(f"✅ Saved PCA metadata")

## 7. Verification

In [ ]:
# Verify saved files
train_verify = pd.read_csv(FEATURES_DIR / 'pca_train.csv')
test_verify = pd.read_csv(FEATURES_DIR / 'pca_test.csv')
pca_verify = joblib.load(MODELS_DIR / 'pca_model.pkl')

print("Verification:")
print(f"  Training shape: {train_verify.shape}")
print(f"  Test shape: {test_verify.shape}")
print(f"  PCA components: {pca_verify.n_components_}")
print(f"  Features (excl. target): {len(train_verify.columns) - 1}")
print(f"\n✅ Notebook 4 Complete!")

In [ ]:
# Summary
print("\n" + "="*50)
print("PCA SUMMARY")
print("="*50)
print(f"Original features: {X_train.shape[1]}")
print(f"Reduced features: {n_components}")
print(f"Variance retained: {sum(pca.explained_variance_ratio_)*100:.2f}%")
print(f"Dimensionality reduction: {(1 - n_components/X_train.shape[1])*100:.1f}%")